In [164]:
import math
import numpy as np
from math import asin, acos, pi, cos, sin, sqrt

In [165]:
def radiansToDegrees(radians):
  return (radians * 180)/pi

def degreesToRadians(degrees):
  return (degrees * pi)/180

def findQuadrant(sine, cosine):
    if cosine > 0 and sine > 0: #1
        return asin(sine)

    if cosine < 0 and sine > 0: #2
        return acos(cosine)

    if cosine < 0 and sine < 0: #3
        return pi - asin(sine)

    if cosine > 0 and sine < 0: #4
        return 2*pi + asin(sine)

In [166]:
#Variable Defintions

a = 1.644324091019978 
e = 0.3839233168996142
Mnot = degreesToRadians(325.5198971624441)
Omega = degreesToRadians(73.19324250923145)
I = degreesToRadians(3.407879729526142)
w = degreesToRadians(319.3189220811686)

timeCurrent = 2459766.5

In [167]:
# Step 1: Mean Motion

meanMotion = 0.0172020989484/sqrt(((a)**3))

In [168]:
#Step 2: Mean Anomaly

meanAnomaly = Mnot + meanMotion * (2459766.5 - 2459800.5)

In [169]:
#Step 3: Eccentric Anomaly

def getE(M):
    Eguess = M
    Mguess = Eguess - e*sin(Eguess)
    while abs(Mguess - M) > 1e-004:
        Mguess = Eguess - e*sin(Eguess)
        Eguess = Eguess - (M - (Eguess - (e*sin(Eguess)))) / (e*cos(Eguess) - 1)
    return Eguess
  
eccentricAnomaly = getE(meanAnomaly)

In [170]:
#Step 4: Calculate r and f


#Cartestian Coordinates
x = a*(cos(eccentricAnomaly) - e)
y = a * math.sqrt(1 - (e**2)) * sin(eccentricAnomaly)
z = 0


r = math.sqrt(x**2 + y**2)
fCos = x/r
fSin = y/r

f = math.atan2(fSin,fCos)

In [171]:
#Step 5: Calculate Ecliptic

vectorEclipticX = r * (cos(f + w) * cos(Omega)  -  cos(I)*sin(f+w)*sin(Omega))
vectorEclipticY = r * (cos(I) * cos(Omega) * sin(f+w) + cos(f+w)*sin(Omega))
vectorEclipticZ = r * sin(I) * sin(f+w)

rArray = np.array([vectorEclipticX, vectorEclipticY, vectorEclipticZ])

In [172]:
#Step 6: Earth -> Sun Vector R

vectorEclipticSunX = -.2395571210776456
vectorEclipticSunY = .9880623728031431
vectorEclipticSunZ = -.00006669698999133755

R = np.array([vectorEclipticSunX, vectorEclipticSunY, vectorEclipticSunZ])

In [173]:
#Step 7: Get the Earth -> Asteroid 
p = R + rArray

In [174]:
#Step 8: Convert Earth -> Asteroid vector (p) to equatorial

pX = p[0]
pY = p[1]*cos(degreesToRadians(23.4379)) - p[2]*sin(degreesToRadians(23.4379))
pZ = p[1]*sin(degreesToRadians(23.4379)) + p[2]*cos(degreesToRadians(23.4379))

In [175]:
pMag = sqrt(pX ** 2 + pY ** 2 + pZ ** 2)

decCoords = asin(pZ/pMag)
cosRA = pX/(pMag * cos(decCoords))
sinRA = pY/(pMag * cos(decCoords))

raCoords = findQuadrant(sinRA, cosRA)

print("Right Acscension: " + str(radiansToDegrees(raCoords)), "Declination: " + str(radiansToDegrees(decCoords)))

Right Acscension: 332.63823837226903 Declination: -18.34326686983059
